<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#函数式编程" data-toc-modified-id="函数式编程-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>函数式编程</a></span><ul class="toc-item"><li><span><a href="#返回函数" data-toc-modified-id="返回函数-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>返回函数</a></span><ul class="toc-item"><li><span><a href="#函数作为返回值" data-toc-modified-id="函数作为返回值-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>函数作为返回值</a></span></li><li><span><a href="#闭包" data-toc-modified-id="闭包-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>闭包</a></span></li><li><span><a href="#练习" data-toc-modified-id="练习-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>练习</a></span></li></ul></li><li><span><a href="#匿名函数" data-toc-modified-id="匿名函数-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>匿名函数</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#练习" data-toc-modified-id="练习-1.2.0.1"><span class="toc-item-num">1.2.0.1&nbsp;&nbsp;</span>练习</a></span></li></ul></li></ul></li><li><span><a href="#装饰器" data-toc-modified-id="装饰器-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>装饰器</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#练习" data-toc-modified-id="练习-1.3.0.1"><span class="toc-item-num">1.3.0.1&nbsp;&nbsp;</span>练习</a></span></li></ul></li></ul></li><li><span><a href="#偏函数" data-toc-modified-id="偏函数-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>偏函数</a></span></li><li><span><a href="#参考" data-toc-modified-id="参考-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>参考</a></span></li></ul></li></ul></div>

![image](http://wx2.sinaimg.cn/thumbnail/69d4185bly1fmf9kfagd3j20ek0ekq88.jpg)

# 函数式编程

## 返回函数
### 函数作为返回值
高阶函数除了可以接受函数作为参数外，还可以把函数作为结果值返回。我们来实现一个可变参数的求和。通常情况下，求和的函数是这样定义的：
```python
def calc_sum(*args):
    ax = 0
    for n in args:
        ax = ax + n
    return ax
```
但是，如果不需要立刻求和，而是在后面的代码中，此时可以不返回求和的结果，而是返回求和的函数，当我们调用函数时，返回的并不是求和结果，而是求和函数

In [1]:
def lazy_sum(*args):
    def sum():
        ax = 0
        for n in args:
            ax = ax + n
        return ax
    return sum

f = lazy_sum(1, 3, 5, 7, 9)
# 返回的是一个函数而不是结果
print(f)

# 调用 f 时返回结果
print(f())

<function lazy_sum.<locals>.sum at 0x000001D617F486A8>
25


在这个例子中，我们在函数`lazy_sum`中又定义了函数`sum`，并且，内部函数`sum`可以引用外部函数`lazy_sum`的参数和局部变量，当`lazy_sum`返回函数`sum`时，相关参数和变量都保存在返回的函数中，这种称为“闭包（Closure）”的程序结构拥有极大的威力。

请再注意一点，当我们调用`lazy_sum()`时，每次调用都会返回一个新的函数，即使传入相同的参数：

In [5]:
f1 = lazy_sum(1, 3, 5, 7, 9)
f2 = lazy_sum(1, 3, 5, 7, 9)
print(f1)
print(f2)
print(f1==f2)
print(f1()==f2())

<function lazy_sum.<locals>.sum at 0x000001D6195D0D08>
<function lazy_sum.<locals>.sum at 0x000001D6195D0E18>
False
True


### 闭包
注意到返回的函数在其定义内部引用了局部变量`args`，所以，当一个函数返回了一个函数后，其内部的局部变量还被新函数引用，所以，闭包用起来简单，实现起来可不容易。

另一个需要注意的问题是，返回的函数并没有立刻执行，而是直到调用了`f()`才执行。我们来看一个例子：

In [6]:
def count():
    fs = []
    for i in range(1, 4):
        def f():
             return i*i
        fs.append(f)
    return fs

f1, f2, f3 = count()

在上面的例子中，每次循环，都创建了一个新的函数，然后，把创建的3个函数都返回了。

你可能认为调用`f1()`，`f2()`和`f3()`结果应该是`1，4，9`，但实际结果是：

In [7]:
print(f1())
print(f2())
print(f3())

9
9
9


原因就在于返回的函数引用了变量i，但它并非立刻执行。等到3个函数都返回时，它们所引用的变量i已经变成了3，因此最终结果为9。 **返回闭包时牢记一点：返回函数不要引用任何循环变量，或者后续会发生变化的变量。**

如果一定要引用循环变量，方法是再创建一个函数，用该函数的参数绑定循环变量当前的值，无论该循环变量后续如何更改，已绑定到函数参数的值不变：

In [8]:
def count():
    def f(j):
        def g():
            return j*j
        return g
    fs = []
    for i in range(1, 4):
        fs.append(f(i)) # f(i)立刻被执行，因此i的当前值被传入f()
    return fs
f1,f2,f3 = count()
print(f1())
print(f2())
print(f3())

1
4
9


### 练习
利用闭包返回一个计数器函数，每次调用它返回递增整数：

In [22]:
def createCounter():
    def nature():
        i = 1
        while True:
            yield i
            i = i +1
    fik = nature()
    def counter():
        return next(fik)
    return counter


# 测试:
counterA = createCounter()
print(counterA(), counterA(), counterA(), counterA(), counterA()) # 1 2 3 4 5
counterB = createCounter()
if [counterB(), counterB(), counterB(), counterB()] == [1, 2, 3, 4]:
    print('测试通过!')
else:
    print('测试失败!')

1 2 3 4 5
测试通过!


另外两种可选方法：
1. 使用列表
```python
def createCounter():
    fs = [0]          #创建一个只有一个元素的列表
    def counter():
        fs[0] = fs[0] + 1
        return fs[0]
    return counter
```

2. 使用 nonlocal 将本地变量变为全局变量
```python
def createCounter():
    n=0
    def f():
        nonlocal n
        n=n+1
        return n
    return f
```

## 匿名函数
python 对匿名函数提供了有限的支持，python 使用 lambda 来创建匿名函数。lambda表达式的特性：
- lambda 只是一个表达式，函数体比 def 简单很多。
- lambda的主体是一个表达式，而不是一个代码块。仅仅能在lambda表达式中封装有限的逻辑进去。
- lambda 函数拥有自己的命名空间，且不能访问自己参数列表之外或全局命名空间里的参数。
- 虽然lambda函数看起来只能写一行，却不等同于C或C++的内联函数，后者的目的是调用小函数时不占用栈内存从而增加运行效率。

形式为：`lambda [arg1 [,arg2,.....argn]]:expression` 

例如在使用 `map()` 计算一个 `list` 中每个数的平方，并生成一个新的 `list` ，就可以使用 lambda 表达式：

In [24]:
list(map(lambda x: x * x, [1, 2, 3, 4, 5, 6, 7, 8, 9]))

[1, 4, 9, 16, 25, 36, 49, 64, 81]

实际上可以看到表达式`lambda x: x * x`，等同于：
```python
def f(x):
    return x * x
```
关键字lambda表示匿名函数，冒号前面的x表示函数参数。

匿名函数有个限制，就是只能有一个表达式，不用写return，返回值就是该表达式的结果。

用匿名函数有个好处，因为函数没有名字，不必担心函数名冲突。此外，匿名函数也是一个函数对象，也可以把匿名函数赋值给一个变量，再利用变量来调用该函数：

In [25]:
f = lambda x: x * x
print(f)
f(4)

<function <lambda> at 0x000001D619693BF8>


16

同样，也可以把匿名函数作为返回值返回，比如：
```python
def build(x, y):
    return lambda: x * x + y * y
```

#### 练习
将现有函数修改为匿名函数

In [27]:
def is_odd(n):
    return n % 2 == 1

L = list(filter(is_odd, range(1, 20)))
print(L)

LL = list(filter(lambda x:x % 2 == 1, range(1, 20)))
print(LL)

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]


## 装饰器
由于函数也是一个对象，而且函数对象可以被赋值给变量，所以，通过变量也能调用该函数。函数对象有一个__name__属性，可以拿到函数的名字。

In [29]:
def now():
    print('2017-12-27')

f = now
f()
# 获取函数的名字
print(now.__name__)
print(f.__name__)

2017-12-27
now
now


假设我们要增强`now()`函数的功能，比如，在函数调用前后自动打印日志，但又不希望修改`now()`函数的定义，这种在代码运行期间动态增加功能的方式，称之为“装饰器”（Decorator）。本质上，decorator就是一个返回函数的高阶函数。所以，我们要定义一个能打印日志的decorator，可以定义如下：

In [30]:
def log(func):
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

观察上面的`log`，因为它是一个`decorator`，所以接受一个函数作为参数，并返回一个函数。我们要借助Python的`@`语法，把decorator置于函数的定义处，调用`now()`函数，不仅会运行`now()`函数本身，还会在运行`now()`函数前打印一行日志：

In [32]:
@log
def now():
    print('2017-12-27')

now()

call now():
2017-12-27


把`@log`放到`now()`函数的定义处，相当于执行了语句：`now = log(now)`。由于`log()`是一个`decorator`，返回一个函数，所以，原来的`now()`函数仍然存在，只是现在同名的`now`变量指向了新的函数，于是调用`now()`将执行新函数，即在`log()`函数中返回的`wrapper()`函数。

`wrapper()`函数的参数定义是`(*args, **kw)`，因此，`wrapper()`函数可以接受任意参数的调用。在`wrapper()`函数内，首先打印日志，再紧接着调用原始函数。

如果`decorator`本身需要传入参数，那就需要编写一个返回`decorator`的高阶函数，写出来会更复杂。比如，要自定义log的文本：

In [37]:
# 三层嵌套函数
def log(text):
    def decorator(func):
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

@log('execute')
def now():
    print('2017-12-27')

now()

execute now():
2017-12-27


和两层嵌套的decorator相比，3层嵌套的效果是这样的：`now = log('execute')(now)` 。我们来剖析上面的语句，首先执行`log('execute')`，返回的是`decorator`函数，再调用返回的函数，参数是`now`函数，返回值最终是`wrapper`函数。

以上两种`decorator`的定义都没有问题，但还差最后一步。因为我们讲了函数也是对象，它有`__name__`等属性，但你去看经过`decorator`装饰之后的函数，它们的`__name__`已经从原来的`'now'`变成了`'wrapper'`：

In [35]:
now.__name__

'wrapper'

因为返回的那个`wrapper()`函数名字就是`'wrapper'`，所以，需要把原始函数的`__name__`等属性复制到`wrapper()`函数中，否则，有些依赖函数签名的代码执行就会出错。

不需要编写`wrapper.__name__ = func.__name__`这样的代码，Python内置的`functools.wraps`就是干这个事的，所以，一个完整的`decorator`的写法如下：

In [ ]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

# 针对带参数的decorator
def log(text):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

#### 练习
设计一个decorator，它可作用于任何函数上，并打印该函数的执行时间：

In [53]:
import time, functools

def metric(fn):
    @functools.wraps(fn)
    def wrapper(*args, **kw):
        start = time.time()
        result = fn(*args,**kw)
        end = time.time()
        print('%s executed in %s ms' % (fn.__name__, end-start))
        return result
    return wrapper

# 测试

@metric
def fast(x, y):
    time.sleep(0.0012)
    return x + y;

@metric
def slow(x, y, z):
    time.sleep(0.1234)
    return x * y * z;

f = fast(11, 22)
print(f)
s = slow(11, 22, 33)
if f != 33:
    print('测试失败!')
elif s != 7986:
    print('测试失败!')

fast executed in 0.002006053924560547 ms
33
slow executed in 0.12429285049438477 ms


## 偏函数
Python的functools模块提供了很多有用的功能，其中一个就是偏函数（Partial function），但这里的偏函数与数学意义上的偏函数并不一样。

在介绍函数参数的时候，我们讲到，通过设定参数的默认值，可以降低函数调用的难度。而偏函数也可以做到这一点。举例如下：

`int()`函数可以把字符串转换为整数，当仅传入字符串时，`int()`函数默认按十进制转换：

In [54]:
int('12345')

12345

但`int()`函数还提供额外的`base`参数，默认值为10。如果传入base参数，就可以做N进制的转换：

In [56]:
# 八进制 转 十进制
print(int('12345', base=8))

# 十六进制 转 十进制
print(int('12345', 16))

5349
74565


假设要转换大量的二进制字符串，每次都传入`int(x, base=2)`非常麻烦，于是，我们想到，可以定义一个`int2()`的函数，默认把base=2传进去,这样，我们转换二进制就非常方便了：

In [57]:
def int2(x, base=2):
    return int(x, base)

print(int2('1000000'))
print(int2('1010101'))

64
85


`functools.partial`就是帮助我们创建一个偏函数的，不需要我们自己定义`int2()`，可以直接使用下面的代码创建一个新的函数int2

In [58]:
import functools
int2 = functools.partial(int, base=2)

print(int2('1000000'))
print(int2('1010101'))

64
85


所以，简单总结`functools.partial`的作用就是，把一个函数的某些参数给固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单。

注意到上面的新的int2函数，仅仅是把base参数重新设定默认值为2，但也可以在函数调用时传入其他值：

In [59]:
int2('1000000', base=10)

1000000

创建偏函数时，实际上可以接收函数对象、`*args`和`**kw`这3个参数，当传入：`int2 = functools.partial(int, base=2)` ，实际上固定了`int()`函数的关键字参数`base`，也就是：`int2('10010')`, 相当于：
```python
kw = { 'base': 2 }
int('10010', **kw)
```
当传入：`max2 = functools.partial(max, 10)`，实际上会把10作为`*args`的一部分自动加到左边，也就是：`max2(5, 6, 7)`，相当于：
```python
args = (10, 5, 6, 7)
max(*args)
```
结果为10。

## 参考
1、[返回函数 - 廖雪峰](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431835236741e42daf5af6514f1a8917b8aaadff31bf000)

2、[匿名函数 - 廖雪芬](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431843456408652233b88b424613aa8ec2fe032fd85a000)

3、[Python3 函数 - 匿名函数 - runoob](http://www.runoob.com/python3/python3-function.html)

4、[装饰器 - 廖雪峰](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/0014318435599930270c0381a3b44db991cd6d858064ac0000#0)

5、[偏函数 - 廖雪峰](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/00143184474383175eeea92a8b0439fab7b392a8a32f8fa000)